<a href="https://colab.research.google.com/github/ananyapillai11/LLM-Movie-Insight-Generator/blob/main/LLM_based_movie_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies and Import Pipelines

In [ ]:
# Install transformers and datasets libraries
!pip install transformers datasets --quiet

# Import Hugging Face pipelines
from transformers import pipeline


Load LLM Pipelines

In [ ]:
# Load a GPT-2 model for text generation (Task 1 & 3)
text_generator = pipeline("text-generation", model="gpt2")

# Load zero-shot classification model for genre prediction (Task 2)
genre_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


Create Movie Dataset

In [ ]:
import pandas as pd

# Sample movie dataset
data = [
    {
        "movie_title": "Skyfall",
        "description": "James Bond's loyalty to M is tested when her past comes back to haunt her.",
        "genres": ["Action", "Thriller"],
        "cast": ["Daniel Craig", "Judi Dench"]
    },
    {
        "movie_title": "Inception",
        "description": "A thief who steals corporate secrets through dream-sharing technology is given a task to plant an idea into a target's mind.",
        "genres": ["Sci-Fi", "Action"],
        "cast": ["Leonardo DiCaprio", "Joseph Gordon-Levitt"]
    },
    {
        "movie_title": "Titanic",
        "description": "A young couple from different social backgrounds fall in love aboard the ill-fated Titanic.",
        "genres": ["Romance", "Drama"],
        "cast": ["Leonardo DiCaprio", "Kate Winslet"]
    },
    {
        "movie_title": "The Dark Knight",
        "description": "Batman sets out to dismantle the remaining criminal organizations but finds himself prey to a rising threat: the Joker.",
        "genres": ["Action", "Crime"],
        "cast": ["Christian Bale", "Heath Ledger"]
    },
    {
        "movie_title": "Forrest Gump",
        "description": "A man with a low IQ recounts the early years of his life when he found himself in the middle of key historical events.",
        "genres": ["Drama", "Romance"],
        "cast": ["Tom Hanks", "Robin Wright"]
    },
    {
        "movie_title": "The Matrix",
        "description": "A hacker learns about the true nature of his reality and his role in the war against its controllers.",
        "genres": ["Sci-Fi", "Action"],
        "cast": ["Keanu Reeves", "Laurence Fishburne"]
    },
    {
        "movie_title": "The Godfather",
        "description": "An organized crime dynasty's aging patriarch transfers control to his reluctant son.",
        "genres": ["Crime", "Drama"],
        "cast": ["Marlon Brando", "Al Pacino"]
    },
    {
        "movie_title": "Interstellar",
        "description": "A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.",
        "genres": ["Sci-Fi", "Adventure"],
        "cast": ["Matthew McConaughey", "Anne Hathaway"]
    },
    {
        "movie_title": "Frozen",
        "description": "When her sister's powers trap their kingdom in eternal winter, Anna teams up with others to save the land.",
        "genres": ["Animation", "Fantasy"],
        "cast": ["Kristen Bell", "Idina Menzel"]
    },
    {
        "movie_title": "Black Panther",
        "description": "T'Challa returns home as king of Wakanda and must defend his country from an old enemy.",
        "genres": ["Action", "Adventure"],
        "cast": ["Chadwick Boseman", "Michael B. Jordan"]
    },
    {
        "movie_title": "Parasite",
        "description": "A poor family schemes to become employed by a wealthy family by infiltrating their household.",
        "genres": ["Thriller", "Drama"],
        "cast": ["Song Kang-ho", "Choi Woo-shik"]
    },
    {
        "movie_title": "The Lion King",
        "description": "A young lion prince flees his kingdom after the death of his father, only to return and reclaim his throne.",
        "genres": ["Animation", "Adventure"],
        "cast": ["Matthew Broderick", "James Earl Jones"]
    },
    {
        "movie_title": "Avengers: Endgame",
        "description": "The Avengers assemble once more to undo Thanos's destruction and restore balance to the universe.",
        "genres": ["Action", "Sci-Fi"],
        "cast": ["Robert Downey Jr.", "Chris Evans"]
    },
    {
        "movie_title": "La La Land",
        "description": "A jazz musician and an aspiring actress fall in love while pursuing their dreams in Los Angeles.",
        "genres": ["Romance", "Musical"],
        "cast": ["Ryan Gosling", "Emma Stone"]
    },
    {
        "movie_title": "Joker",
        "description": "A failed comedian's descent into madness leads to the birth of Gotham’s criminal mastermind.",
        "genres": ["Crime", "Drama"],
        "cast": ["Joaquin Phoenix", "Robert De Niro"]
    },
    {
        "movie_title": "Coco",
        "description": "A young boy embarks on a journey to the Land of the Dead to find his great-great-grandfather.",
        "genres": ["Animation", "Family"],
        "cast": ["Anthony Gonzalez", "Gael García Bernal"]
    },
    {
        "movie_title": "Get Out",
        "description": "A Black man uncovers disturbing secrets when he meets his white girlfriend’s family.",
        "genres": ["Horror", "Thriller"],
        "cast": ["Daniel Kaluuya", "Allison Williams"]
    },
    {
        "movie_title": "The Social Network",
        "description": "The story of Facebook's founding and the legal battles that followed.",
        "genres": ["Biography", "Drama"],
        "cast": ["Jesse Eisenberg", "Andrew Garfield"]
    },
    {
        "movie_title": "The Grand Budapest Hotel",
        "description": "A concierge and his protégé become involved in a series of events surrounding a valuable painting.",
        "genres": ["Comedy", "Adventure"],
        "cast": ["Ralph Fiennes", "Tony Revolori"]
    },
    {
        "movie_title": "Shutter Island",
        "description": "Two U.S. Marshals investigate a psychiatric facility on an isolated island and uncover shocking secrets.",
        "genres": ["Mystery", "Thriller"],
        "cast": ["Leonardo DiCaprio", "Mark Ruffalo"]
    }
]

# Convert data to a DataFrame for processing
df = pd.DataFrame(data)
df.head()


,movie_title,description,genres,cast
0,Skyfall,James Bond's loyalty to M is tested when her p...,"[Action, Thriller]","[Daniel Craig, Judi Dench]"
1,Inception,A thief who steals corporate secrets through d...,"[Sci-Fi, Action]","[Leonardo DiCaprio, Joseph Gordon-Levitt]"
2,Titanic,A young couple from different social backgroun...,"[Romance, Drama]","[Leonardo DiCaprio, Kate Winslet]"
3,The Dark Knight,Batman sets out to dismantle the remaining cri...,"[Action, Crime]","[Christian Bale, Heath Ledger]"
4,Forrest Gump,A man with a low IQ recounts the early years o...,"[Drama, Romance]","[Tom Hanks, Robin Wright]"


Task 1 – Generate Creative Descriptions

In [ ]:
import json

generated_descriptions = []

# Generate a new creative description using GPT-2 for each movie
for index, row in df.iterrows():
  # Formulate prompt for text generation
    prompt = f"Movie title: {row['movie_title']}\nPlot: {row['description']}\nNew creative description:"
    new_desc = text_generator(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
    # Store results
    result = {
        "movie_title": row["movie_title"],
        "original_description": row["description"],
        "generated_description": new_desc.strip(),
        "genres": row["genres"],
        "cast": row["cast"]
    }
    generated_descriptions.append(result)

# Save generated descriptions to JSON file
with open("generated_descriptions.json", "w") as f:
    json.dump(generated_descriptions, f, indent=2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/

Task 2 – Predict Genres Using Zero-Shot Classification & Evaluate Accuracy

In [ ]:
from transformers import pipeline
import pandas as pd

# Expanded candidate labels based on all genres in the dataset
candidate_labels = [
    "Action", "Drama", "Comedy", "Romance", "Horror", "Thriller",
    "Sci-Fi", "Fantasy", "Crime", "Adventure", "Animation",
    "Mystery", "Biography", "Musical", "Family",
    "Romantic Comedy", "Sci-Fi Thriller", "Superhero", "Psychological Thriller"
]


# Load the data
df = pd.DataFrame(data)

# Initialize variables for accuracy calculation
correct_predictions = 0
total_predictions = 0
genre_results = []

# Zero-shot genre classification using BART model
for index, row in df.iterrows():
    desc = row["description"]
    actual_genres = [g.strip().title() for g in row["genres"]]  # Normalize casing

    # Predict top 5 genres using zero-shot classifier
    prediction = genre_classifier(desc, candidate_labels)
    predicted_genres = prediction['labels'][:5]

    # Check for overlap between predicted and actual genres
    overlap = set(predicted_genres) & set(actual_genres)
    if overlap:
        correct_predictions += 1
    total_predictions += 1

    # Store result for review
    genre_results.append({
        "movie_title": row["movie_title"],
        "actual_genres": actual_genres,
        "predicted_genres": predicted_genres,
        "overlap": list(overlap),
        "match_found": bool(overlap)
    })

# Print final accuracy
accuracy = correct_predictions / total_predictions
print(f"Genre Prediction Accuracy (Top 5): {accuracy * 100:.2f}%")

# View predictions as DataFrame
genre_df = pd.DataFrame(genre_results)
genre_df.head(20)


Genre Prediction Accuracy (Top 5): 80.00%


,movie_title,actual_genres,predicted_genres,overlap,match_found
0,Skyfall,"[Action, Thriller]","[Drama, Crime, Action, Biography, Family]",[Action],True
1,Inception,"[Sci-Fi, Action]","[Crime, Mystery, Adventure, Action, Sci-Fi]","[Action, Sci-Fi]",True
2,Titanic,"[Romance, Drama]","[Romance, Action, Adventure, Biography, Animat...",[Romance],True
3,The Dark Knight,"[Action, Crime]","[Superhero, Crime, Action, Adventure, Drama]","[Action, Crime]",True
4,Forrest Gump,"[Drama, Romance]","[Adventure, Biography, Action, Mystery, Family]",[],False
5,The Matrix,"[Sci-Fi, Action]","[Action, Adventure, Mystery, Sci-Fi Thriller, ...",[Action],True
6,The Godfather,"[Crime, Drama]","[Crime, Family, Action, Adventure, Biography]",[Crime],True
7,Interstellar,"[Sci-Fi, Adventure]","[Sci-Fi, Adventure, Action, Sci-Fi Thriller, F...","[Adventure, Sci-Fi]",True
8,Frozen,"[Animation, Fantasy]","[Family, Action, Adventure, Drama, Mystery]",[],False
9,Black Panther,"[Action, Adventure]","[Action, Adventure, Superhero, Drama, Biography]","[Adventure, Action]",True


Task 3 – Generate Plot Twists

In [ ]:
from transformers import pipeline
import json
import pandas as pd
import re

# Load the GPT-2 text generator
text_generator = pipeline("text-generation", model="gpt2", temperature=0.9, top_k=50, top_p=0.92, do_sample=True)

plot_twists = []
# Helper function to remove repeated sentences
def clean_text(text):
    # Remove repeated sentences
    sentences = re.split(r'(?<=[.!?]) +', text)
    seen = set()
    unique = []
    for s in sentences:
        s_clean = s.strip()
        if s_clean and s_clean not in seen:
            seen.add(s_clean)
            unique.append(s_clean)
    return ' '.join(unique)

# Generate plot twists
for index, row in df.iterrows():
    prompt = (
        f"Movie: {row['movie_title']}\n"
        f"Plot: {row['description']}\n"
        "Write a shocking and clever plot twist or alternate ending in 2-3 sentences:"
    )

    twist_raw = text_generator(prompt, max_length=150, num_return_sequences=1)[0]['generated_text']
    twist_cleaned = clean_text(twist_raw.split("ending in 2-3 sentences:")[-1].strip())

    plot_twists.append({
        "movie_title": row["movie_title"],
        "plot_twist": twist_cleaned
    })

# Save to JSON
with open("plot_twists.json", "w") as f:
    json.dump(plot_twists, f, indent=2)


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transform

Download JSON Outputs

In [ ]:
# Download generated output files
from google.colab import files
files.download("generated_descriptions.json")
files.download("plot_twists.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>